In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2799171378644884982, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3697891876328075740
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [16]:
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
from keras.applications import MobileNet

bottleneck_final_model=MobileNet(weights="imagenet", include_top=False, pooling = "avg")

In [17]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "MobileNet_descriptors"

In [18]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [19]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [20]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [21]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [22]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [23]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [24]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [25]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [26]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_2': 0.4, 'lr': 0.001, 'beta_1': 0.5}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 4s - loss: 0.6865 - acc: 0.7787 - val_loss: 1.2108 - val_acc: 0.6611
Epoch 2/15
 - 4s - loss: 0.6838 - acc: 0.8224 - val_loss: 1.3429 - val_acc: 0.6648
Epoch 3/15
 - 4s - loss: 0.7219 - acc: 0.8307 - val_loss: 1.6584 - val_acc: 0.6601
Epoch 4/15
 - 4s - loss: 0.7848 - acc: 0.8358 - val_loss: 1.7135 - val_acc: 0.6781
Epoch 5/15
 - 4s - loss: 0.8164 - acc: 0.8342 - val_loss: 1.8056 - val_acc: 0.6599
Epoch 6/15
 - 4s - loss: 0.8621 - acc: 0.8352 - val_loss: 1.8355 - val_acc: 0.6699
Epoch 7/15
 - 4s - loss: 0.8748 - acc: 0.8382 - val_loss: 1.9442 - val_acc: 0.6459
Epoch 00007: early stopping
14958/14958 [==============================] - 1s 34us/step
Validation Accuracy: 64.5942%
Validation Loss: 1.9442279749278075
Test Accuracy: 0.7884813669170717
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_

Epoch 6/15
 - 4s - loss: 0.5079 - acc: 0.8249 - val_loss: 1.0695 - val_acc: 0.6452
Epoch 7/15
 - 4s - loss: 0.4938 - acc: 0.8302 - val_loss: 1.0843 - val_acc: 0.6419
Epoch 8/15
 - 4s - loss: 0.4879 - acc: 0.8365 - val_loss: 1.0589 - val_acc: 0.6534
Epoch 9/15
 - 4s - loss: 0.4803 - acc: 0.8401 - val_loss: 1.0694 - val_acc: 0.6538
Epoch 10/15
 - 4s - loss: 0.4691 - acc: 0.8446 - val_loss: 1.1335 - val_acc: 0.6459
Epoch 11/15
 - 4s - loss: 0.4669 - acc: 0.8479 - val_loss: 1.1186 - val_acc: 0.6586
Epoch 12/15
 - 4s - loss: 0.4616 - acc: 0.8504 - val_loss: 1.1277 - val_acc: 0.6549
Epoch 13/15
 - 4s - loss: 0.4626 - acc: 0.8517 - val_loss: 1.1283 - val_acc: 0.6598
Epoch 14/15
 - 4s - loss: 0.4591 - acc: 0.8535 - val_loss: 1.1089 - val_acc: 0.6665
Epoch 15/15
 - 4s - loss: 0.4566 - acc: 0.8571 - val_loss: 1.1855 - val_acc: 0.6582
14958/14958 [==============================] - 1s 35us/step
Validation Accuracy: 65.8243%
Validation Loss: 1.1854938022900956
Test Accuracy: 0.7906171748416556
*_**

Test Accuracy: 0.2566651936956842
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.6, 'lr': 0.01, 'beta_1': 0.7}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.1634 - acc: 0.6972 - val_loss: 1.6593 - val_acc: 0.5560
Epoch 2/15
 - 4s - loss: 1.3978 - acc: 0.7053 - val_loss: 2.0821 - val_acc: 0.5469
Epoch 3/15
 - 4s - loss: 1.5451 - acc: 0.7045 - val_loss: 2.2791 - val_acc: 0.5280
Epoch 4/15
 - 4s - loss: 1.5561 - acc: 0.7153 - val_loss: 2.5224 - val_acc: 0.4858
Epoch 00004: early stopping
14958/14958 [==============================] - 1s 36us/step
Validation Accuracy: 48.5827%
Validation Loss: 2.5224029991014327
Test Accuracy: 0.6716747680070703
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2'

Epoch 3/15
 - 4s - loss: 1.8470 - acc: 0.2981 - val_loss: 1.6155 - val_acc: 0.3472
Epoch 4/15
 - 4s - loss: 1.6964 - acc: 0.3450 - val_loss: 1.5664 - val_acc: 0.3389
Epoch 5/15
 - 4s - loss: 1.5853 - acc: 0.3830 - val_loss: 1.5282 - val_acc: 0.3307
Epoch 6/15
 - 4s - loss: 1.4858 - acc: 0.4199 - val_loss: 1.4968 - val_acc: 0.3337
Epoch 00006: early stopping
14958/14958 [==============================] - 1s 38us/step
Validation Accuracy: 33.3668%
Validation Loss: 1.4968013336416521
Test Accuracy: 0.5197378111651201
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.4, 'lr': 1e-06, 'beta_1': 0.5}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 2.1951 - acc: 0.1700 - val_loss: 1.7570 - val_acc: 0.1473
Epoch 2/15
 - 4s - loss: 1.9601 - acc: 0.2278 - val_loss: 1.6722 - val_acc: 0.2639
Epoch 3/15
 - 4s - loss: 1.7960 - acc: 0.2793 - 

Test Accuracy: 0.7920901458241273
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.999, 'lr': 0.0001, 'beta_1': 0.2}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.9264 - acc: 0.6507 - val_loss: 1.1270 - val_acc: 0.5162
Epoch 2/15
 - 4s - loss: 0.6532 - acc: 0.7633 - val_loss: 1.0314 - val_acc: 0.5971
Epoch 3/15
 - 4s - loss: 0.5840 - acc: 0.7897 - val_loss: 0.9842 - val_acc: 0.6191
Epoch 4/15
 - 4s - loss: 0.5396 - acc: 0.8067 - val_loss: 0.9929 - val_acc: 0.6203
Epoch 5/15
 - 4s - loss: 0.5127 - acc: 0.8167 - val_loss: 0.9559 - val_acc: 0.6404
Epoch 6/15
 - 4s - loss: 0.4911 - acc: 0.8230 - val_loss: 0.9695 - val_acc: 0.6399
Epoch 7/15
 - 4s - loss: 0.4718 - acc: 0.8297 - val_loss: 0.9754 - val_acc: 0.6413
Epoch 8/15
 - 4s - loss: 0.4563 - acc: 0.8355 - val_loss: 0.9508 - val_acc: 0.6491
Epoch 9/15
 - 4s - loss: 0.4451 -

...

Parameters testing:  {'beta_2': 0.999, 'lr': 0.0001, 'beta_1': 0.8}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 20s - loss: 0.9179 - acc: 0.6591 - val_loss: 1.1443 - val_acc: 0.5264
Epoch 2/15
 - 9s - loss: 0.6578 - acc: 0.7614 - val_loss: 1.0492 - val_acc: 0.5854
Epoch 3/15
 - 8s - loss: 0.5864 - acc: 0.7875 - val_loss: 1.0133 - val_acc: 0.6069
Epoch 4/15
 - 9s - loss: 0.5437 - acc: 0.8062 - val_loss: 1.0025 - val_acc: 0.6185
Epoch 5/15
 - 9s - loss: 0.5145 - acc: 0.8131 - val_loss: 0.9705 - val_acc: 0.6315
Epoch 6/15
 - 9s - loss: 0.4891 - acc: 0.8241 - val_loss: 0.9609 - val_acc: 0.6482
Epoch 7/15
 - 9s - loss: 0.4714 - acc: 0.8297 - val_loss: 0.9311 - val_acc: 0.6515
Epoch 8/15
 - 9s - loss: 0.4590 - acc: 0.8345 - val_loss: 0.9771 - val_acc: 0.6388
Epoch 9/15
 - 9s - loss: 0.4447 - acc: 0.8409 - val_loss: 0.9481 - val_acc: 0.6579
Epoch 10/15
 - 8s - loss: 0.4328 - acc: 0.8442 - val_loss: 0.9661 - val_acc: 0.6510
Epoch 11/15
 - 9s - loss: 0.4234 - acc: 0.8476 - v

 - 9s - loss: 0.4318 - acc: 0.8445 - val_loss: 0.9744 - val_acc: 0.6512
Epoch 11/15
 - 9s - loss: 0.4200 - acc: 0.8494 - val_loss: 0.9757 - val_acc: 0.6566
Epoch 12/15
 - 9s - loss: 0.4119 - acc: 0.8524 - val_loss: 0.9872 - val_acc: 0.6494
Epoch 13/15
 - 9s - loss: 0.4017 - acc: 0.8553 - val_loss: 0.9560 - val_acc: 0.6685
Epoch 14/15
 - 9s - loss: 0.3946 - acc: 0.8586 - val_loss: 0.9644 - val_acc: 0.6631
Epoch 15/15
 - 9s - loss: 0.3850 - acc: 0.8605 - val_loss: 0.9796 - val_acc: 0.6648
14958/14958 [==============================] - 2s 156us/step
Validation Accuracy: 66.4795%
Validation Loss: 0.9795563052283748
Test Accuracy: 0.7954043305346885
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.4, 'lr': 0.1, 'beta_1': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 20s - loss: 11.3200 - acc: 0.2947 - val_loss: 13.5458 - val_acc: 0.15

Epoch 8/15
 - 9s - loss: 1.3957 - acc: 0.4566 - val_loss: 1.5344 - val_acc: 0.3162
Epoch 9/15
 - 9s - loss: 1.3397 - acc: 0.4799 - val_loss: 1.5078 - val_acc: 0.3243
Epoch 10/15
 - 9s - loss: 1.2956 - acc: 0.4961 - val_loss: 1.4846 - val_acc: 0.3319
Epoch 11/15
 - 9s - loss: 1.2459 - acc: 0.5183 - val_loss: 1.4632 - val_acc: 0.3381
Epoch 12/15
 - 9s - loss: 1.2112 - acc: 0.5371 - val_loss: 1.4439 - val_acc: 0.3447
Epoch 13/15
 - 9s - loss: 1.1762 - acc: 0.5513 - val_loss: 1.4267 - val_acc: 0.3500
Epoch 14/15
 - 9s - loss: 1.1425 - acc: 0.5629 - val_loss: 1.4116 - val_acc: 0.3553
Epoch 15/15
 - 9s - loss: 1.1200 - acc: 0.5733 - val_loss: 1.3971 - val_acc: 0.3611
14958/14958 [==============================] - 2s 129us/step
Validation Accuracy: 36.1145%
Validation Loss: 1.3971252494483792
Test Accuracy: 0.5782147591692444
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing

Epoch 8/15
 - 9s - loss: 0.4599 - acc: 0.8349 - val_loss: 0.9662 - val_acc: 0.6407
Epoch 9/15
 - 9s - loss: 0.4449 - acc: 0.8398 - val_loss: 0.9572 - val_acc: 0.6502
Epoch 10/15
 - 9s - loss: 0.4336 - acc: 0.8447 - val_loss: 0.9434 - val_acc: 0.6577
Epoch 11/15
 - 9s - loss: 0.4243 - acc: 0.8497 - val_loss: 0.9932 - val_acc: 0.6495
Epoch 12/15
 - 9s - loss: 0.4114 - acc: 0.8528 - val_loss: 0.9618 - val_acc: 0.6530
Epoch 13/15
 - 10s - loss: 0.4024 - acc: 0.8580 - val_loss: 1.0001 - val_acc: 0.6473
Epoch 00013: early stopping
14958/14958 [==============================] - 2s 136us/step
Validation Accuracy: 64.7279%
Validation Loss: 1.0001403356220278
Test Accuracy: 0.785609073501252
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.999, 'lr': 0.0001, 'beta_1': 0.6}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 0.9279 - acc: 

Test Accuracy: 0.7884813669170717
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.999, 'lr': 0.001, 'beta_1': 0.7}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 0.6362 - acc: 0.7709 - val_loss: 1.0514 - val_acc: 0.6015
Epoch 2/15
 - 9s - loss: 0.4924 - acc: 0.8222 - val_loss: 1.0534 - val_acc: 0.6375
Epoch 3/15
 - 9s - loss: 0.4485 - acc: 0.8377 - val_loss: 0.9933 - val_acc: 0.6701
Epoch 4/15
 - 9s - loss: 0.4266 - acc: 0.8453 - val_loss: 1.0584 - val_acc: 0.6524
Epoch 5/15
 - 9s - loss: 0.4066 - acc: 0.8528 - val_loss: 0.9645 - val_acc: 0.6730
Epoch 6/15
 - 9s - loss: 0.3934 - acc: 0.8585 - val_loss: 1.1505 - val_acc: 0.6673
Epoch 7/15
 - 9s - loss: 0.3792 - acc: 0.8621 - val_loss: 1.1085 - val_acc: 0.6632
Epoch 8/15
 - 9s - loss: 0.3728 - acc: 0.8681 - val_loss: 1.0277 - val_acc: 0.6782
Epoch 9/15
 - 10s - loss: 0.3623 